In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [50.4 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,932 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [18]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [19]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [20]:
# 2. Create a temporary view of the DataFrame.

# Add the CSV file to SparkFiles
spark.sparkContext.addFile(url)

# Read the CSV into a DataFrame
home_sales_df = spark.read.option("header", "true").csv(SparkFiles.get("home_sales_revised.csv"))

# Create a Temporary Table from the DataFrame
home_sales_df.createOrReplaceTempView("home_sales")

# Perform a query and show only the first 5 rows
result = spark.sql("SELECT * FROM home_sales")
result.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [21]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# SQL query to find the mean price of 4-bedroom homes for every year, formatted to display trailing zeros and a dollar sign

# SQl query code
query = """
SELECT YEAR(date) AS year_sold,
       CONCAT('$', format_number(AVG(price), 2)) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query
avg_price_by_year_4_bedrooms = spark.sql(query)

# Show the results
avg_price_by_year_4_bedrooms.show()

+---------+-----------+
|year_sold|  avg_price|
+---------+-----------+
|     2019|$300,263.70|
|     2020|$298,353.78|
|     2021|$301,819.44|
|     2022|$296,363.88|
+---------+-----------+



In [22]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# SQL query to find the average price of homes with 3 bedrooms and 3 bathrooms for each year built,
# formatted to display with a dollar sign and two decimal places, including trailing zeros.

# SQl query code
query = """
SELECT date_built AS year_built,
       CONCAT('$', format_number(ROUND(AVG(price), 2), 2)) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
avg_price_by_year_built = spark.sql(query)

# Show the results
avg_price_by_year_built.show()

+----------+-----------+
|year_built|  avg_price|
+----------+-----------+
|      2010|$292,859.62|
|      2011|$291,117.47|
|      2012|$293,683.19|
|      2013|$295,962.27|
|      2014|$290,852.27|
|      2015|$288,770.30|
|      2016|$290,555.07|
|      2017|$292,676.79|
+----------+-----------+



In [23]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# SQL query to find the average price of homes with specific characteristics for each year built,
# formatted to display with a dollar sign and two decimal places, including trailing zeros.

# SQl query code
query = """
SELECT date_built AS year_built,
       CONCAT('$', format_number(ROUND(AVG(price), 2), 2)) AS avg_price
FROM home_sales
WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
avg_price_by_year_built_specifics = spark.sql(query)

# Show the results
avg_price_by_year_built_specifics.show()

+----------+-----------+
|year_built|  avg_price|
+----------+-----------+
|      2010|$285,010.22|
|      2011|$276,553.81|
|      2012|$307,539.97|
|      2013|$303,676.79|
|      2014|$298,264.72|
|      2015|$297,609.97|
|      2016|$293,965.10|
|      2017|$280,317.58|
+----------+-----------+



In [58]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Make sure the DataFrame is not currently cached
home_sales_df.unpersist()

# Measure the time taken to execute the query without caching

# Query Start time
start_time_uncached = time.time()

# SQL query on uncached table
spark.sql("""
SELECT view,
       CONCAT('$', format_number(ROUND(AVG(price), 2), 2)) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT) DESC
""").show(100)

# Query end time and calculate difference
end_time_uncached = time.time()
uncached_runtime = end_time_uncached - start_time_uncached

# Display SQL query runtime
print(f"Uncached runtime: {uncached_runtime} seconds")

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
|  79|$1,009,565.08|
|  78|$1,080,649.37|
|  77|$1,076,205.56|
|  76|$1,058,802.78|
|  75|$1,114,042.94|
|  74|  $745,077.00|
|  73|  $752,861.18|
|  72|  $780,914.67|
|  71|  $775,651.10|
|  70|  $695,865.58|
|  69|  $750,537.94|
|  68|  $716,785.44|
|  67|  $737,970.96|
|  66|  $712,475.00|
|  65|  $736,679.93|
|  64|  $767,036.67|
|  63|  $711,614.55|
|  62|  $759,150.14|
|  61|  $746,877.59|
|  60|  $754,939.65|
|  59|  $791,453.00|
|  58|  $759,764.65|
|  57|  $734,340.50|
|  56|  $718,

In [59]:
# 7. Cache the the temporary table home_sales.

# home_sales_df is the DataFrame used to create the cached temporary table 'home_sales'
home_sales_df.cache()

# Create a temporary table (view) from the cached DataFrame
home_sales_df.createOrReplaceTempView('home_sales')

In [60]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [62]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Measure the time taken to execute the query with caching

# Query start time
start_time_cached = time.time()

# SQL query on cached table
spark.sql("""
SELECT view,
       CONCAT('$', format_number(ROUND(AVG(price), 2), 2)) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT) DESC
""").show(100)

# Query end time and calculate difference
end_time_cached = time.time()
cached_runtime = end_time_cached - start_time_cached

# Display SQL query runtime and difference in cached vs uncached runtime
print(f"Cached runtime: {cached_runtime} seconds")

print(f"Runtime difference querying cached table (vs. uncached table; positive numbers indicate runtime improvement): {uncached_runtime - cached_runtime} seconds")

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
|  79|$1,009,565.08|
|  78|$1,080,649.37|
|  77|$1,076,205.56|
|  76|$1,058,802.78|
|  75|$1,114,042.94|
|  74|  $745,077.00|
|  73|  $752,861.18|
|  72|  $780,914.67|
|  71|  $775,651.10|
|  70|  $695,865.58|
|  69|  $750,537.94|
|  68|  $716,785.44|
|  67|  $737,970.96|
|  66|  $712,475.00|
|  65|  $736,679.93|
|  64|  $767,036.67|
|  63|  $711,614.55|
|  62|  $759,150.14|
|  61|  $746,877.59|
|  60|  $754,939.65|
|  59|  $791,453.00|
|  58|  $759,764.65|
|  57|  $734,340.50|
|  56|  $718,

In [63]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Path to save the partitioned Parquet files
output_path = "/home_sales_partitioned_by_date_built"

# Write the DataFrame to Parquet, partitioned by 'date_built'
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet(output_path)

In [65]:
# 11. Read the parquet formatted data.

# Read the partitioned Parquet data back into a DataFrame
partitioned_df = spark.read.parquet(output_path)

# Show the DataFrame schema to verify the partitioning
partitioned_df.printSchema()

# Perform a sample query to demonstrate the use of partitioning
sample_year = "2013"
partitioned_df.where(partitioned_df.date_built == sample_year).show()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|7530a2d8-1ae3-451...|2021-06-13| 379628|       2|        2|       2235|   14384|     1|         0|  23|      2013|
|d6b9fc56-fc83-469...|2021-10-23| 332675|       3|        2|       1515|    8390|     1|         0| 

In [66]:
# 12. Create a temporary table for the parquet data.

# partitioned_df is our DataFrame loaded from the partitioned Parquet files
partitioned_df.createOrReplaceTempView('home_sales')

# Use Spark SQL to query the data
spark.sql("SELECT * FROM home_sales LIMIT 10").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [67]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# Query start time
start_time_parquet = time.time()

# SQL query on parquet DataFrame
spark.sql("""
SELECT view,
       CONCAT('$', format_number(ROUND(AVG(price), 2), 2)) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT) DESC
""").show(100)

# Query end time and calculate difference
end_time_parquet = time.time()
parquet_runtime = end_time_parquet - start_time_parquet

# Display SQL query runtime and difference in Parquet vs cached runtime
print(f"Parquet runtime: {parquet_runtime} seconds")
print(f"Runtime difference querying Parquet formatted table (vs. cached table; positive numbers indicate runtime improvement): {cached_runtime - parquet_runtime} seconds")

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
|  79|$1,009,565.08|
|  78|$1,080,649.37|
|  77|$1,076,205.56|
|  76|$1,058,802.78|
|  75|$1,114,042.94|
|  74|  $745,077.00|
|  73|  $752,861.18|
|  72|  $780,914.67|
|  71|  $775,651.10|
|  70|  $695,865.58|
|  69|  $750,537.94|
|  68|  $716,785.44|
|  67|  $737,970.96|
|  66|  $712,475.00|
|  65|  $736,679.93|
|  64|  $767,036.67|
|  63|  $711,614.55|
|  62|  $759,150.14|
|  61|  $746,877.59|
|  60|  $754,939.65|
|  59|  $791,453.00|
|  58|  $759,764.65|
|  57|  $734,340.50|
|  56|  $718,

In [68]:
# 14. Uncache the home_sales temporary table.
home_sales_df.unpersist()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [69]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False